<a href="https://colab.research.google.com/github/thu4n/WiFi-Vision-Counting/blob/main/data-processing/truecount_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/thu4n/WiFi-Vision-Counting.git

Cloning into 'WiFi-Vision-Counting'...
remote: Enumerating objects: 1438, done.
remote: Counting objects: 100% (603/603), done.
remote: Compressing objects: 100% (335/335), done.
remote: Total 1438 (delta 248), reused 493 (delta 172), pack-reused 835 (from 1)
Receiving objects: 100% (1438/1438), 130.66 MiB | 18.06 MiB/s, done.
Resolving deltas: 100% (551/551), done.


In [4]:
import os
os.chdir('/content/WiFi-Vision-Counting/real-time-counting')
!pwd

/content/WiFi-Vision-Counting/real-time-counting


In [41]:
import os
import cv2
import time
import argparse
import csv
import torch
import model.detector
import utils.utils

In [39]:
cfg = utils.utils.load_datafile('data/coco.data')
weight = '/content/WiFi-Vision-Counting/real-time-counting/modelzoo/yolofv2-nano-190-epoch-0.953577ap-model.pth'
output_dir = 'tmp'
os.makedirs(output_dir, exist_ok=True)

In [42]:
    # Load the model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.detector.Detector(cfg["classes"], cfg["anchor_num"], True).to(device)
    model.load_state_dict(torch.load(weight, map_location=device))
    model.eval()

load param...


<ipython-input-42-909db69c0bd9>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(weight, map_location=device))


Detector(
  (backbone): ShuffleNetV2(
    (first_conv): Sequential(
      (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (stage2): Sequential(
      (0): ShuffleV2Block(
        (branch_main): Sequential(
          (0): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(24, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=24, bias=False)
          (4): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (5): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (6): BatchNorm2d(24, eps=1e-05, momentum=0.1, a

In [43]:
# Load label names
LABEL_NAMES = []
with open(cfg["names"], 'r') as f:
    for line in f.readlines():
        LABEL_NAMES.append(line.strip())
print(len(LABEL_NAMES))

80


In [61]:
true_count = 0
date = '051024'
session = 'session1'
input_dir = f'/content/drive/MyDrive/UIT_Graduation_Thesis/Dataset/0_Raw/2024-10-05/Image_Frames/session_1/{true_count}_person'
image_dir = f'{output_dir}/{date}_{session}_{true_count}'
os.makedirs(image_dir, exist_ok=True)

In [62]:
total = 0
# Process each image in the directory
for img_file in os.listdir(input_dir):
    img_path = os.path.join(input_dir, img_file)
    ori_img = cv2.imread(img_path)
    raw = ori_img.copy()
    if ori_img is None:
        # print(f"Skipping file {img_file} (not an image)")
        continue

    # Data preprocessing
    res_img = cv2.resize(ori_img, (cfg["width"], cfg["height"]), interpolation=cv2.INTER_LINEAR)
    img = res_img.reshape(1, cfg["height"], cfg["width"], 3)
    img = torch.from_numpy(img.transpose(0, 3, 1, 2)).to(device).float() / 255.0

    # Model inference
    start = time.perf_counter()
    preds = model(img)
    end = time.perf_counter()
    inference_time = (end - start) * 1000.0
    print(f"{img_file} - Forward time: {inference_time:.2f} ms")

    # Post-process predictions
    output = utils.utils.handel_preds(preds, cfg, device)
    output_boxes = utils.utils.non_max_suppression(output, conf_thres=0.3, iou_thres=0.4)

    h, w, _ = ori_img.shape
    scale_h, scale_w = h / cfg["height"], w / cfg["width"]

    person_count = 0
    confidence_scores = []

    # Draw bounding boxes and collect predictions for persons
    for box in output_boxes[0]:
        box = box.tolist()
        obj_score = box[4]
        category = LABEL_NAMES[int(box[5])]

        # Check if the detected object is a person
        if category.lower() == "person":
            person_count += 1
            confidence_scores.append(obj_score)

            x1, y1 = int(box[0] * scale_w), int(box[1] * scale_h)
            x2, y2 = int(box[2] * scale_w), int(box[3] * scale_h)

            cv2.rectangle(ori_img, (x1, y1), (x2, y2), (255, 255, 0), 2)
            cv2.putText(ori_img, f'{obj_score:.2f}', (x1, y1 - 5), 0, 0.7, (0, 255, 0), 2)
            cv2.putText(ori_img, category, (x1, y1 - 25), 0, 0.7, (0, 255, 0), 2)

    # Save result image
    if person_count == true_count:
        total += 1
        cv2.imwrite(image_dir + '/' + img_file, raw)

print(f'Total {true_count} person: {total}')

frame_1728097500.1987903.jpg - Forward time: 90.55 ms
frame_1728097500.3069067.jpg - Forward time: 38.95 ms
frame_1728097500.4141593.jpg - Forward time: 54.47 ms
frame_1728097500.522867.jpg - Forward time: 61.52 ms
frame_1728097500.6313818.jpg - Forward time: 44.82 ms
frame_1728097500.7402372.jpg - Forward time: 96.26 ms
frame_1728097500.848001.jpg - Forward time: 46.40 ms
frame_1728097500.955437.jpg - Forward time: 46.02 ms
frame_1728097501.0629613.jpg - Forward time: 27.68 ms
frame_1728097501.1708663.jpg - Forward time: 66.66 ms
frame_1728097501.2799733.jpg - Forward time: 28.39 ms
frame_1728097501.3875315.jpg - Forward time: 27.44 ms
frame_1728097501.4951246.jpg - Forward time: 34.76 ms
frame_1728097501.6024294.jpg - Forward time: 30.21 ms
frame_1728097501.7102897.jpg - Forward time: 40.10 ms
frame_1728097501.818989.jpg - Forward time: 28.08 ms
frame_1728097501.9267027.jpg - Forward time: 81.65 ms
frame_1728097502.0341651.jpg - Forward time: 54.53 ms
frame_1728097502.1422713.jpg - F

In [63]:
!cp -r /content/WiFi-Vision-Counting/real-time-counting/tmp/051024_session1_1/ /content/drive/MyDrive/UIT_Graduation_Thesis/Dataset/1_Processed/CV_Origin_TrueCount/

In [56]:
# !rm -rf /content/drive/MyDrive/UIT_Graduation_Thesis/Dataset/1_Processed/CV_Origin_TrueCount/051024_session1_3

In [49]:
# !rm -rf /content/WiFi-Vision-Counting/real-time-counting/tmp/051024_session1_2/*

In [26]:
# !mkdir /content/drive/MyDrive/UIT_Graduation_Thesis/Dataset/1_Processed/CV_Origin_TrueCount

In [25]:
# !mkdir /content/WiFi-Vision-Counting/real-time-counting/tmp/051024_session1_5